In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import datetime
import io
import os
import sys
import cv2

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
os.environ['PYOPENGL_PLATFORM'] = 'osmesa'

import pose_pipeline
pose_pipeline.set_environmental_variables()  # allows running the pose pipeline
from pose_pipeline.utils.jupyter import play
from pose_pipeline.pipeline import *

# load this specific schema
from pose_pipeline.demo.h36_dj import *

# NOTE: adjust to your installation
os.environ['CDF_LIB'] = '/home/jcotton/CDF/cdf38_0-dist/src/lib'

In [ ]:
# Use this script https://github.com/anibali/h36m-fetch to download human3.6 and unpack it. 
# Please follow the Human 3.6 license.

video_project = 'h36m'
human36_extracted_directory = '/mnt/data0/datasets/human3.6/h36m-fetch/extracted'

In [ ]:
# Import all the data into PosePipe
import_data(human36_extracted_directory)

In [ ]:
# show frame from random video
key = (Video & 'video_project="h36m"').fetch('KEY')[0]
video = (Video & key).fetch1('video')
cap = cv2.VideoCapture(video)
ret, frame = cap.read()
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
cap.release()
os.delete(video)

In [ ]:
plt.figure()
plt.imshow(frame)
plt.plot(poses2d[0, :, 0], poses2d[0, :, 1], '.')

# Now process them with pipeline code

Alternative can use 'scripts/process_h36m.py' to run from CLI and easily run multiple scripts on different GPUs.

In [ ]:
from pose_pipeline.utils.standard_pipelines import top_down_pipeline
#james_walking_sync_20220830_173523 james_walking_sync_20220830_173351
#homeleg_stairs_20220901_120001

VIDEO_PROJECT = "h36m"
keys = (Video & f'video_project="{VIDEO_PROJECT}"').fetch('KEY')

for k in keys:
    #top_down_pipeline(k, top_down_method_name="OpenPose", tracking_method_name='DeepSortYOLOv4')
    top_down_pipeline(k, top_down_method_name="MMPoseHalpe", tracking_method_name='DeepSortYOLOv4')
    #top_down_pipeline(k, top_down_method_name="MMPoseHalpe")

# Annotation (when multiple people detected)

Check for unannotated videos and run annotation gui. Then rerun the above cell.

In [ ]:
# Create visualizations to check assignment for ones not automatically done

filt = 'video_project="h36m"'

BlurredVideo.populate((TrackingBbox - PersonBbox) & filt, suppress_errors=False)
TrackingBboxVideo.populate(BlurredVideo - PersonBbox & filt, suppress_errors=True)

In [ ]:
# Now run GUI

from pose_pipeline.utils.annotation import annotate

to_label = ((TrackingBboxVideo & filt) - PersonBboxValid).fetch('KEY')
print(f'{len(to_label)} found to annotate')

key = to_label[0]

annotate(key)